In [1]:
import numpy as np
import pandas as pd
import requests
import json
import dotenv
import os

## Getting Biographical Data from the Congress API

The most important skill is reading the API documentaation efficiently.

Look ONLY for these five things:

1. Root: the begining part of all URLs associated with this API for getting data back (I often find this in example first)

https://api.congress.gov/v3

2. Endpoint: the second part of the URL, generally it points to a specific data set to be returned

/member/{bioguideID}

The curly braces mean:fill this in for wichever member of congress you want

3. The headers: data that WE give to the ApI to identity ourselves, and provide othe information if requested
 * User Agent string: a string that identifies your software. If you are writing a Bot, what is you bot's name, and what methids are you using to access this API.

 The standard convertion is:

 'botname/version (email) python-requests/request-package-version'

In [15]:
botname = 'targ'
version = '0.0'
email = 'wna6mj@virginia.edu'
useragent = f'{botname}/{version} ({email}) python-requests/{requests.__version__}'
headers = {'User-Agent':useragent}
headers

{'User-Agent': 'targ/0.0 (wna6mj@virginia.edu) python-requests/2.32.5'}

4. API Parameters: there are ways to contro; what data is requested. The parameters are generally listed in the documentation for each endpoint. 3 kinds of parameters:

* Query parameters: should be listed in a dictionary and passed to the `params` argument of ` requests.get()`

* Path parameter: are part of the endpoint (such as bioguideId)

* Global parameters: apply to any use fo the API regardless if endpoint. Often that includes your API key. Can be in eith the headers ot the params

5. If an API key is needed, how to get one, and how will you let the API know the key?

API keys should be saved in a .env file in the same folder as your code. Use `dotenv` package to load the key into your code without ever displaying/exposing your key.

In [30]:
dotenv.load_dotenv()
congresskey =  os.getenv('congresskey')

In [29]:
params = {'format': 'json',
          'api_key': congresskey}

Put is all togethere

In [27]:
bioguideId = 'M001239'
root = 'https://api.congress.gov/v3'
endpoint = f'/member/{bioguideId}'

r = requests.get(root + endpoint,
                 headers = headers,
                 params = params)

r

<Response [200]>

In [28]:
r.json()

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'officeAddress': '1013 Longworth House Office Building',
   'phoneNumber': '(202) 225-4711',
   'zipCode': 20515},
  'bioguideId': 'M001239',
  'birthYear': '1968',
  'cosponsoredLegislation': {'count': 166,
   'url': 'https://api.congress.gov/v3/member/M001239/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': 'Image courtesy of the Member',
   'imageUrl': 'https://www.congress.gov/img/member/67744ba20b34857ecc909149_200.jpg'},
  'directOrderName': 'John J. McGuire III',
  'district': 5,
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'McGuire, John J.',
  'lastName': 'McGuire',
  'middleName': 'J.',
  'officialWebsiteUrl': 'https://mcguire.house.gov/',
  'partyHistory': [{'partyAbbreviation': 'R',
    'partyName': 'Republican',
    'startYear': 2025}],
  'previousNames': [{'directOrderName': 'John McGuire',
    'endDate': '2025-01-10T17:36:18Z',
   